In [1]:
import pandas as pd
import numpy as np
import gc
import logging
from tqdm import tqdm
from config.database import HOST, PORT, USER, PASSWORD, DATABASE

import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import types as sqltype
from config.database import HOST, PORT, USER, PASSWORD, DATABASE

psql_engine = create_engine("postgresql://"+USER+":"+PASSWORD+"@"+HOST+":"+str(PORT)+"/"+DATABASE)

In [2]:
#with psql_engine.connect() as conn:
#    sql = "DELETE FROM project WHERE proj_id LIKE '% %';"
#    conn.execute(sql)

In [3]:
def escape(s):
    return (s).replace("'", "''").replace('%', '%%').replace('<','').replace('>','')

In [10]:
with psql_engine.connect() as conn:
    df_commiter = pd.read_sql("SELECT proj_id, commiter_aliase_id FROM commit;", con= conn)
    df_author = pd.read_sql("SELECT proj_id, author_aliase_id FROM commit;", con= conn)
    
    commiters = df_commiter['commiter_aliase_id'].unique()
    authors = df_author['author_aliase_id'].unique()

    all_commit_aliase = pd.Series(np.append(commiters, authors)).drop_duplicates()
    all_commit_aliase.describe()


In [12]:
all_commit_aliase.describe()

count        4451
unique       4451
top       DBlasby
freq            1
dtype: object

In [15]:
with psql_engine.connect() as conn:
    df_commiter_svn = pd.read_sql("SELECT * FROM aliase WHERE source='SVN';", con= conn)

In [16]:
df_commiter_svn['aliase_id'].describe()

count          723
unique         723
top       tonyfang
freq             1
Name: aliase_id, dtype: object

***


***

In [6]:
with psql_engine.connect() as conn:
    df = pd.read_sql("SELECT thread_id FROM thread WHERE thread_type='email' ;", con= conn)
    #df = pd.read_sql("SELECT * FROM project WHERE proj_id LIKE '% %';", con= conn)

In [7]:
df
#for id in df['proj_id'].values:
#    print(id)

,thread_id
0,gdal#114#003f01c95f02$28a8bc50$79fa34f0$%%robe...
1,gdal#11#C82ADCB6.17F91%%james.e.hopper@saic.com
2,gdal#9#C82ADCB6.17F91%%james.e.hopper@saic.com
3,gdal#14#C82AE221.17F93%%james.e.hopper@saic.com
4,gdal#18#C82B09FC.17FA4%%james.e.hopper@saic.com
...,...
4362,proj#37#BFCD579C.6299%%ovv@hetnet.nl
4363,proj#66#BFCE22EE.62A6%%ovv@hetnet.nl
4364,proj#38#BFCE22EE.62A6%%ovv@hetnet.nl
4365,gdal#124#BD9EFB41.1111%%lorenzo.sassolini@maio...


In [9]:
with psql_engine.connect() as conn:
    count_num = 0
    for thisid in tqdm(df['thread_id'].values) :
#        sql = """ DELETE FROM thread WHERE thread_id='%s'""" %(escape(thisid))
        conn.execute(sql)
        count_num+= 1
        #if not(count_num%1000) :
        #    print("Delete:", count_num)


100%|██████████| 4367/4367 [08:56<00:00,  8.14it/s]


In [4]:
df[df.where(df["thread_id"]=="fdo-internals#1#D20FC5C02CA4AB41891CFE76D91C57A9384F6D8C4E@ADSK-NAMSG-02.MGDADSK.autodesk.com")].drop_duplicates()

,thread_id
0,NaN


In [11]:
with psql_engine.connect() as conn:
    count_num = 0
    for thisid in df['thread_id'].values :
        sql = """ DELETE FROM thread WHERE thread_id='%s'""" %(escape(thisid))
        conn.execute(sql)
        count_num+= 1
        if not(count_num%1000) :
            print("Delete:", count_num)


In [15]:
projects = df["thread_id"].apply(lambda x: x.split('#')[0]).sort_values(key=lambda x: x.str.len()).unique()
for proj in projects:
    print(proj)

issues
geowave
geonode
actinia
e__geos
om__qgis
com__qgis
team engine
mobility db
qgis desktop
Unicode error
Build is broken.
jts topology suite
Extension mechanisms
django-osgeo-importer
Configurable Importer
668603944@web.de__qgis
Ignore non-layer files.
48BAFC99.901@o2.pl__qgis
53FD8B70.902@gmx.ch__qgis
4F72D1BC.406@met.no__proj
4E9D582F.600@c-s.fr__qgis
497D8D42.106@lqt.it__qgis
53AED6C0.805@gmx.ch__qgis
53F237D2.109@gmx.ch__qgis
50E5514D.600@gmx.ch__qgis
48209673.40903@o2.pl__qgis
5507F90D.0@carto.net__qgis
48197E7C.40405@o2.pl__qgis
46AFBF11.40706@o2.pl__qgis
4822F864.3@pobox.com__proj
47A3AC6A.70506@o2.pl__qgis
485FA678.10706@o2.pl__qgis
5569CE62.4@gmail.com__qgis
4594FBDA.70508@o2.pl__qgis
4802F1F5.20706@o2.pl__qgis
446F1281.90906@alex3__geos
486A99F4.90703@o2.pl__qgis
48AC0DDE.40509@o2.pl__qgis
48063690.20004@o2.pl__qgis
4A60E59D.407@acm.org__geos
451D99F7.90908@o2.pl__qgis
49DB9BA6.40609@lqt.it__qgis
48A08CD4.200@frii.com__geos
441149B1.20705@tkk.fi__geos
53CFD4B9.30003@gmx.c